In [2]:
import pandas as pd
import dill
import codecs
import pyautogui
import time
import pyperclip
from datetime import datetime
from pyperiscope import Scope
from pyperiscope import Pilot

In [13]:
with pd.HDFStore('film_database.h5') as store:
    films = store['films']
    production_co = store['production_company']
    production_co_link = store['production_co_link']
    producers = store['producers']
    producers_link = store['producers_link']
    emails = store['emails']
    agents = store['agents']
    agents_link = store['agents_link']

In [14]:
# convert Updated values from string to datetime
films['updated'] = pd.to_datetime(films['updated'], format="mixed")

# set and filter cutoff
cutoff_date = datetime(2023, 1, 1)
films = films[films['updated'] >= cutoff_date].copy()

# set of film_id's that are left
valid_film_ids = set(films.index)

In [15]:
# Filter the producers_link table to keep only links to valid films
producers_link = producers_link[producers_link['film_id'].isin(valid_film_ids)].copy()
# Get the IDs of producers that still have at least one film
valid_producer_ids = set(producers_link['producer_id'])

In [28]:
production_co_link = production_co_link[production_co_link['film_id'].isin(valid_film_ids)].copy()
valid_production_co = set(production_co_link['production_co_id'])

In [32]:
# Filter the producers table to keep only valid producers
producers = producers.loc[producers.index.isin(valid_producer_ids)].copy()
# Filter the production_co table by producer index
production_co = production_co.loc[production_co.index.isin(valid_production_co)].copy()

In [36]:
emails=emails.drop(emails.index)
agents=agents.drop(agents.index)
agents_link=agents_link.drop(agents_link.index)

In [38]:
with pd.HDFStore('film_database.h5', mode='r+') as store:
    store.put('films', films)
    store.put('production_co_link', production_co_link)
    store.put('production_company', production_co)
    store.put('producers',producers)
    store.put('producers_link', producers_link)
    store.put('emails', emails)
    store.put('agents', agents)
    store.put('agents_link', agents_link)